### Days: wps_climdex_days

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) days functions

- [climdex.su](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=25) Computes the SU (summer days) climdexindex: the annual count of days where daily maximum temperature exceeds 25 degreesCelsius.
- [climdex.id](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=11) Computes the ID (icing days) climdexindex: the annual count of days where daily maximum temperature was below 0 degrees Celsius.

In [ ]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url, local_path

In [ ]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [ ]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

In [ ]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [ ]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_days?

#### Run wps_climdex_days process for climdex.su (summer days)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",        
            days_type="summer",
            vector_name="summer_days",
            output_path=output_file.name,
        )
su_url = output.get()[0]

#### Run wps_climdex_days process for climdex.id (icing days)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="icing_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",        
            days_type="icing",
            vector_name="icing_days",
            output_path=output_file.name,
        )
id_url = output.get()[0]

In [ ]:
# Method for tests
def test_days_output(url, vector_name, expected_file, expected_vector_name):    
    with NamedTemporaryFile(
        suffix=".rda", prefix="tmp_copy", dir="/tmp", delete=True
    ) as tmp_file:
        urlretrieve(url, tmp_file.name)
        robjects.r(f"load(file='{tmp_file.name}')")
        
    output_vector = robjects.r(vector_name)

    robjects.r(
        "load(file='{}')".format(resource_filename("tests",f"data/{expected_file}"))
    )
    expected_vector = robjects.r(expected_vector_name)

    for index in range(len(expected_vector)):
        assert str(output_vector[index]) == str(expected_vector[index])

#### Test output against expected output

In [ ]:
test_days_output(
        su_url, "summer_days", "expected_summer_days.rda", "expected_summer_days"
    )
test_days_output(
        id_url, "icing_days", "expected_icing_days.rda", "expected_icing_days"
    )